In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 1. Get Device for Training

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# 2. Define the Class

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Creating Instance

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Pass input data

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


# 3. Model Layers

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


convert input image to contiguous array (minibatch dimension (at dim=0) is maintained

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## Linear Layer (applies a linear transformation on the input using stored weights and biases

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## ReLU
Non-Linear activation function for complex mapping between input and output
Applied after linear transformations.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2712,  0.1207, -0.8477, -0.7979, -0.0828, -0.1495,  0.0856,  0.3118,
         -0.2872,  0.1660,  0.1289, -0.2128,  0.3383,  0.2528,  0.1031,  0.0645,
          0.6710, -0.6279,  0.2049,  0.0064],
        [ 0.0511,  0.0863, -0.5740, -0.3119, -0.1551, -0.2211,  0.1796,  0.1680,
         -0.2608, -0.0732,  0.2027, -0.2738,  0.1337, -0.1445, -0.0528, -0.3814,
          0.5525, -0.3246, -0.1844,  0.0554],
        [ 0.4052, -0.1317, -0.8415, -0.4743, -0.1364, -0.0420, -0.1494,  0.4182,
         -0.0156,  0.2454,  0.3935, -0.0255, -0.0043,  0.2053,  0.1123, -0.2306,
          0.5183, -0.0284, -0.3039,  0.1516]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2712, 0.1207, 0.0000, 0.0000, 0.0000, 0.0000, 0.0856, 0.3118, 0.0000,
         0.1660, 0.1289, 0.0000, 0.3383, 0.2528, 0.1031, 0.0645, 0.6710, 0.0000,
         0.2049, 0.0064],
        [0.0511, 0.0863, 0.0000, 0.0000, 0.0000, 0.0000, 0.1796, 0.1680, 0.0000,
         0.0000, 0.2027, 0.0000, 0.1337, 0.0000, 0.00

## nn.Sequential
ordered container of modules.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# 4. Model Parameters

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0203,  0.0051,  0.0050,  ...,  0.0040,  0.0294,  0.0309],
        [-0.0111,  0.0016,  0.0220,  ...,  0.0137,  0.0350,  0.0035]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0019,  0.0275], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0315, -0.0319, -0.0017,  ...,  0.0365, -0.0090,  0.0259],
        [ 0.0187, -0.0141, -0.0225,  ...,  0.0437, -0.0081, -0.0120]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 